# 10 Convolutional Layers

*IMPORTANT*

If you're experiencing issues with this notebook, please update your ```cuDNN``` library:

```
conda activate tf
pip install nvidia-cudnn-cu11==8.9.0.131
```

Once that's done, import the libraries that we are going to use - no surprises so far:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as image
import tensorflow as tf
from tensorflow import keras

In the networks we have considered so far, all pixels of the input image data have been directly connected to the neurons of the hidden layer. This is fine as long as we are dealing with low-resolution image data, but as soon as we get to higher-quality input data, computational costs rapidly explode. In addition, the network gets even more sensitive to slight variations in the position of features even. Two kinds of special network layers exist to deal with these issues: Convolutional layers and pooling layers. 

Convolutional layers put the C into CNNs and can be depicted as little windows that slide over the input image and apply a small filter kernel matrix (typically) that extracts details that are relevant for the classification. In Tensorflow, we only need to provide the size of the filter kernel matrix as well as the number of filters that shall be applied. The best set of filter matrix elements is learned during the training process.

![Convolutional layers](cnn.jpeg "Title")
Source: https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53

The ``scipy`` library contains a ``convolve`` function that allows us to easily apply some generic filter kernels with various effects:

In [ ]:
# %conda install scipy
from scipy.ndimage import convolve

# A nice selection of kernels: https://en.wikipedia.org/wiki/Kernel_(image_processing)
identity = np.array([[0, 0, 0],[0, 1, 0], [0, 0, 0]])
edge = np.array([[1, 0, -1],[0, 0, 0],[-1, 0, 1]])
sharpen = np.array([[0, -1, 0],[-1, 5, -1],[0, -1, 0]])
gaussian = np.array([[1, 2, 1],[2, 4, 2],[1, 2, 1]]) * 0.0625

What could be more generic than a cat image? Here, we do some extra steps to display the image in its full pixel size assuming a screen pixel density of 100 dpi:

In [ ]:
img = np.array(image.open('cat.png').convert('L'))

dpi = 100
height, width = img.shape
figsize = width / dpi, height / dpi

fig = plt.figure(figsize=figsize)
plt.imshow(img, cmap='gray')
plt.show()

As an example, let's apply the Gaussian blur filter to the cat image: 

In [ ]:
blurImg = convolve(img, gaussian)
blurImg = convolve(blurImg, gaussian)
fig = plt.figure(figsize=figsize)
plt.imshow(blurImg, cmap='gray')
plt.show()

Because of the small size of the filter kernel, the softening effect is quite subtle. Nevertheless, the cat's whiskers and fur are clearly even softer than before. The ``sharpen`` kernel, on the other hand, results in the opposite effect: contrasts and edges are getting more prominent while smooth areas remain unchanged:

In [ ]:
fig = plt.figure(figsize=figsize)
plt.imshow(convolve(img, sharpen), cmap='gray')
plt.show()

Back to neural networks: While we just applied a given filter kernel to a given image, a convolutional layer within a CNN does not have fixed kernel matrix elements. Rather, the network is trained to find which combination of elements works best for the dataset for feature extraction. While this may sound almost too good to be true, there is a hefty price that we are paying. Namely, the number of neural network parameters increases dramatically as does the time for training.

Therefore, after a convolutional layer, one typically applies a pooling layer (MaxPooling or AveragePooling) to reduce the complexity of the network by combining several nodes and only considering either the average or the maximum of their outputs, resulting in a dramatic reduction of data. We will apply these two new layers to a new dataset. That is, a collection of galaxy images that we will use to train a network to distinguish between three basic shapes: Elliptical, spiral and irregular galaxies.

The data itself is a pre-selected subset of the EFIGI survey dataset:

https://www.astromatic.net/projects/efigi

First off, we need to get the images into the notebook. All relevant information is stored in the ``efigi.dat`` file:

In [ ]:
data = open("galaxies/efigi.dat","r")

names = []
types = []

for line in data:
    fields = line.split(" ")
    names.append( fields[0] )
    types.append( fields[1] )
    
nData = len(names)
imgSize = 128

Elliptical galaxies belong to class 0, spirals to class 1 and irregulars to class 2. Now we can create the data arrays and iterate through the folder:

In [ ]:
galaxies = np.zeros((nData,imgSize,imgSize,3))
labels = np.zeros(nData, dtype='int')

for i in range(nData):
    filename = "galaxies/png/"+str(names[i])+".png"
    img = image.open(filename)

    imgResized = img.resize(size=(imgSize,imgSize))
    imgArr = np.array(imgResized)

    imgArr = imgArr/255.
    
    galaxies[i,:,:,:] = imgArr 
    labels[i] = types[i]

Do we need to flip the input data range?

In [ ]:
plt.hist(galaxies.flatten(), 100)
plt.show()

Next we need to split the full dataset into a training and a test set. Here, index 955 marks the transition:

In [ ]:
split = 955

trainGalaxies = galaxies[:split,:,:,:]
trainLabels = labels[:split]

testGalaxies = galaxies[split+1:nData-1,:,:,:]
testLabels = labels[split+1:nData-1]

To avoid having to go through all this again, we store the data as ``*.npz`` files:

In [ ]:
np.save("galaxies/trainGalaxies.npy", trainGalaxies)
np.save("galaxies/trainLabels.npy", trainLabels)

np.save("galaxies/testGalaxies.npy", testGalaxies)
np.save("galaxies/testLabels.npy", testLabels)

We are ready to configure the network. The configuration we are using is based on a conference proceeding by an astrophysics group from Egypt (Khalifa et al 2017, https://arxiv.org/abs/1709.02245). We start with a feature extraction layer that uses a convolutional layer with 96 filters (8x8 filter kernel each) and a pooling layer that bins every 3x3 pixels to a single output:

In [ ]:
galNet = keras.Sequential([
    keras.layers.Conv2D(96, (6, 6), activation='relu', input_shape=(imgSize,imgSize,3)),
    keras.layers.MaxPooling2D(pool_size=(3,3)),
    keras.layers.Flatten(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

Let's take a look at the structure of our network:

In [ ]:
galNet.summary()

Time to compile the model and train it. After every training epoch, the network performance is evalulated and stored.

In [ ]:
galNet.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = galNet.fit(trainGalaxies, trainLabels, validation_data=(testGalaxies, testLabels), epochs=50)

The history object contains a history attribut that stores some of the training metrics:

In [ ]:
print(dir(history))
print(type(history.history))
print(history.history.keys())

Let's evaluate the training process by looking at a plot of the evolution of the performance:

In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel("Epochs")
plt.ylabel("Test set performance")
plt.plot(history.history['val_accuracy'])
plt.savefig("galnet_performance.pdf")
plt.show()

Apparently, we're getting into the overfit regime. Are there ways to deal with that and train complex networks even further? More on that next week ...

In [ ]:
img = image.open("galaxies/ngc1232b.jpg")

imgResized = img.resize(size=(imgSize,imgSize))
imgArr = np.array(imgResized)

imgArr = imgArr/255.

plt.imshow(imgArr)
plt.show()

In [ ]:
imgArrExp = np.expand_dims(imgArr,axis=0)
print(imgArrExp.shape)

In [ ]:
pred = galNet.predict(imgArrExp)
print(pred)